In [3]:
import scraping_class
import pandas as pd
awards = 'awards.txt'
connector = scraping_class.Connector(awards)

In [7]:
from bs4 import BeautifulSoup
import requests

url = 'https://www.imdb.com/name/nm0000093/awards?ref_=nm_awd'
response = connector.get(url, 'awards')

html = response[0].text
soup = BeautifulSoup(html,'html.parser')


In [174]:
#Locates and parse the node
table_node = soup.find_all('table', attrs ={'class': 'awards'})
table_node

[<table cellpadding="5" cellspacing="0" class="awards">
 <tr>
 <td align="center" class="award_year" rowspan="1">
 <a href="/event/ev0000003/2016"> 2016
 </a> </td>
 <td class="award_outcome" rowspan="1">
 <b>Nominee</b><br/>
 <span class="award_category">Oscar</span>
 </td>
 <td class="award_description">
   Best Motion Picture of the Year<br/>
 <a href="/title/tt1596363">The Big Short</a> <span class="title_year">(2015)</span>
 <br/>
 <div class="shared_with">Shared with:</div>
 <ul>
 <a href="/name/nm0306890">Dede Gardner</a>
 <br/>
 <a href="/name/nm1250070">Jeremy Kleiner</a>
 <br/>
 </ul>
 </td>
 </tr>
 <tr>
 <td align="center" class="award_year" rowspan="1">
 <a href="/event/ev0000003/2014"> 2014
 </a> </td>
 <td class="award_outcome" rowspan="1">
 <b>Winner</b><br/>
 <span class="award_category">Oscar</span>
 </td>
 <td class="award_description">
   Best Motion Picture of the Year<br/>
 <a href="/title/tt2024544">12 Years a Slave</a> <span class="title_year">(2013)</span>
 <br/

In [195]:
awards_data = []
for i in table_node:
    try:
        #actor = i.find('h3').text.strip('\n')
        award_year = i.find_all('td', attrs ={'class': 'award_year'})
        award_year =[i.find('a').text.strip('\n') for i in award_year]
        outcome = i.find_all('td', attrs = {'class': 'award_outcome'})
        
        outcome = [i.text for i in outcome]
        
        award = [i.split('\n')[2] for i in outcome]
        result = [i.split('\n')[1] for i in outcome]
        
        description = i.find('td', attrs ={'class': 'award_description'}).text.split('\n')
        
        awards_data.append([award_year, award, result, description[1]])
    except:
        NameError
awards_data

[[[' 2016', ' 2014', ' 2012', ' 2009', ' 1996'],
  ['Oscar', 'Oscar', 'Oscar', 'Oscar', 'Oscar'],
  ['Nominee', 'Winner', 'Nominee', 'Nominee', 'Nominee'],
  '  Best Motion Picture of the Year'],
 [[' 2012', ' 2009', ' 2007', ' 1996', ' 1995'],
  ['Golden Globe',
   'Golden Globe',
   'Golden Globe',
   'Golden Globe',
   'Golden Globe'],
  ['Nominee', 'Nominee', 'Nominee', 'Winner', 'Nominee'],
  '  Best Performance by an Actor in a Motion Picture - Drama'],
 [[' 2015', ' 2014', ' 2002'],
  ['Primetime Emmy', 'Primetime Emmy', 'Primetime Emmy'],
  ['Nominee', 'Winner', 'Nominee'],
  '  Outstanding Television Movie'],
 [[' 2016', ' 2014', ' 2012', ' 2009', ' 2007'],
  ['BAFTA Film Award',
   'BAFTA Film Award',
   'BAFTA Film Award',
   'BAFTA Film Award',
   'BAFTA Film Award'],
  ['Nominee', 'Winner', 'Nominee', 'Nominee', 'Nominee'],
  '  Best Film'],
 [[' 2016'], ['Felix'], ['Nominee'], '  Best Supporting Actor'],
 [[' 2016', ' 2014', ' 2012'],
  ['AACTA International Award',
   'A

In [245]:
COLS = ['award year', 'award', 'outcome', 'category']
df = pd.DataFrame(data=awards_data, columns= COLS)

mask = df.award.apply(lambda award: 'Oscar' in award)
df1

,award year,award,outcome,category
0,"[ 2016, 2014, 2012, 2009, 1996]","[Oscar, Oscar, Oscar, Oscar, Oscar]","[Nominee, Winner, Nominee, Nominee, Nominee]",Best Motion Picture of the Year
